In [42]:
import re
from pypdf import PdfReader, PdfWriter

pattern1 = re.compile(r'^[0-9](?:\.[0-9]+)+ [A-Z].*', re.M)
pattern2 = re.compile(r'Contents|CONTENTS')

reader = PdfReader('exp.pdf')

In [30]:
headings = {}

for page_number, page in enumerate(reader.pages):
    print(page_number)
    text = page.extract_text()
    if pattern2.search(text) != None:
        if pattern2.search(text).group() == 'Contents':
            headings[text.split('\n')[1]] = page_number
    else:
        for i in pattern1.findall(text):
            headings[i] = page_number

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
backup = headings.copy()

In [ ]:
print(headings)

heading_tree = {}

def gen_key(head:str):
    temp = head.split()
    key = temp[0].split('.')
    key = [int(i) for i in key]
    while len(key) < 3:
        key.append(0)
    return tuple(key)

for head in headings:
    heading_tree[gen_key(head)] = head

{'1 Introduction 3': 12, '1.1 Machine Perception': 14, '1.2 An Example': 14, '1.2.1 Related ﬁelds': 22, '1.3 The Sub-problems of Pattern Classiﬁcation': 22, '1.3.1 Feature Extraction': 22, '1.3.2 Noise': 23, '1.3.3 Overﬁtting': 23, '1.3.4 Model Selection': 23, '1.3.5 Prior Knowledge': 23, '1.3.6 Missing Features': 24, '1.3.7 Mereology': 24, '1.3.8 Segmentation': 24, '1.3.9 Context': 25, '1.3.10 Invariances': 25, '1.3.11 Evidence Pooling': 26, '1.3.12 Costs and Risks': 26, '1.3.13 Computational Complexity': 27, '1.4 Learning and Adaptation': 27, '1.4.1 Supervised Learning': 27, '1.4.2 Unsupervised Learning': 28, '1.4.3 Reinforcement Learning': 28, '1.5 Conclusion': 28, '2 Bayesian decision theory 3': 36, '2.1 Introduction': 38, '2.2 Bayesian Decision Theory – Continuous Fea-': 42, '2.2.1 Two-CategoryClassiﬁcation': 43, '2.3 Minimum-Error-RateClassiﬁcation': 44, '2.4 Classiﬁers, Discriminant Functions and Deci-': 48, '2.4.1 TheMulti-CategoryCase': 48, '2.4.2 TheTwo-CategoryCase': 49, '2.

ValueError: invalid literal for int() with base 10: 'A'

In [41]:
heading_tree

{(1, 0, 0): '1 Introduction 3',
 (1, 1, 0): '1.1 Machine Perception',
 (1, 2, 0): '1.2 An Example',
 (1, 2, 1): '1.2.1 Related ﬁelds',
 (1, 3, 0): '1.3 The Sub-problems of Pattern Classiﬁcation',
 (1, 3, 1): '1.3.1 Feature Extraction',
 (1, 3, 2): '1.3.2 Noise',
 (1, 3, 3): '1.3.3 Overﬁtting',
 (1, 3, 4): '1.3.4 Model Selection',
 (1, 3, 5): '1.3.5 Prior Knowledge',
 (1, 3, 6): '1.3.6 Missing Features',
 (1, 3, 7): '1.3.7 Mereology',
 (1, 3, 8): '1.3.8 Segmentation',
 (1, 3, 9): '1.3.9 Context',
 (1, 3, 10): '1.3.10 Invariances',
 (1, 3, 11): '1.3.11 Evidence Pooling',
 (1, 3, 12): '1.3.12 Costs and Risks',
 (1, 3, 13): '1.3.13 Computational Complexity',
 (1, 4, 0): '1.4 Learning and Adaptation',
 (1, 4, 1): '1.4.1 Supervised Learning',
 (1, 4, 2): '1.4.2 Unsupervised Learning',
 (1, 4, 3): '1.4.3 Reinforcement Learning',
 (1, 5, 0): '1.5 Conclusion',
 (2, 0, 0): '2 Bayesian decision theory 3',
 (2, 1, 0): '2.1 Introduction',
 (2, 2, 0): '2.2 Bayesian Decision Theory – Continuous Fea-'

In [ ]:
writer = PdfWriter()

def dfs_add_label(node, level):
    if level > 2:
        return

    global parent
    old_parent = parent

    list_node = list(node)
    list_node[level] += 1

    while tuple(list_node) in heading_tree:
        head = heading_tree[tuple(list_node)]
        pg_no = headings[head]
        parent = writer.add_outline_item(head, pg_no, parent=parent)
        dfs_add_label(tuple(list_node), level+1)
        list_node[level] += 1
    
    parent = old_parent

parent = None

for ch in range(1,10):
    temp = (ch, 0, 0)
    head = heading_tree[temp]
    pg_no = headings[head]
    parent = writer.add_outline_item(head, pg_no)
    dfs_add_label(temp, 1)

with open('final.pdf', 'wb') as output:
    writer.write(output)

IndexError: list index out of range

In [39]:
gen_key('1 Introduction')

[1, 0, 0]

In [25]:
page = reader.pages[104]

# print(page.extract_text())
# print(pattern2.search(page.extract_text()))
# print(pattern2.search(page.extract_text()).group())
print(str(page.extract_text()).split('\n'))
print(re.findall(r'^\d.*', page.extract_text()))

['Contents', '3 Maximum likelihood and Bayesian estimation 3', '3.1 Introduction . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3', '3.2 Maximum Likelihood Estimation . . . . . . . . . . . . . . . . . . . . . 4', '3.2.1 The General Principle . . . . . . . . . . . . . . . . . . . . . . . 4', '3.2.2 The Gaussian Case:Unknownµ ................. 7', '3.2.3 The Gaussian Case:Unknown µ and Σ ............. 7', '3.2.4 Bias . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8', '3.3 Bayesian estimation . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9', '3.3.1 The Class-Conditional Densities . . . . . . . . . . . . . . . . . 10', '3.3.2 The Parameter Distribution . . . . . . . . . . . . . . . . . . . . 10', '3.4 Bayesian Parameter Estimation:Gaussian Case . . . . . . . . . . . . . 11', '3.4.1 The Univariate Case: p(µ|D) ................... 1 1', '3.4.2 The Univariate Case: p(x|D) ................... 1 4', '3.4.3 The Multivariate Case . . . . . . . . . .